# Rules:
Before running the code, please be sure to do the following:
1. Edit the stand-alone table "New Charter Location" with the name and desired address
2. Paths can be changed, but not the names of output features

In [1]:
from arcgis.gis import GIS
import arcpy, os, sys
aprx=arcpy.mp.ArcGISProject("CURRENT")
p=arcpy.mp.ArcGISProject("CURRENT")
m= p.listMaps()[0]
arcpy.env.overwriteOutput=True

# Making the School of Focus

In [3]:
#geocode new school location
# Log in to Portal
gis=GIS("home")

arcpy.geocoding.GeocodeAddresses(
    in_table="NewCharterLocation",
    address_locator="https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/Esri World Geocoder",
    in_address_fields="'Address or Place' Address VISIBLE NONE;Address2 <None> VISIBLE NONE;Address3 <None> VISIBLE NONE;Neighborhood <None> VISIBLE NONE;City City VISIBLE NONE;County <None> VISIBLE NONE;State State VISIBLE NONE;ZIP ZIP VISIBLE NONE;ZIP4 <None> VISIBLE NONE;Country <None> VISIBLE NONE",
    out_feature_class=r"C:\Users\Avigail.Vargas\Documents\ArcGIS\Projects\PersonalProjects\3MileRadiusTemplate\Default.gdb\NewCharterLocationPoint",
    out_relationship_type="STATIC",
    country="US",
    location_type="ADDRESS_LOCATION",
    category=None,
    output_fields="ALL"
)

<Result 'C:\\Users\\Avigail.Vargas\\Documents\\ArcGIS\\Projects\\PersonalProjects\\3MileRadiusTemplate\\Default.gdb\\NewCharterLocationPoint'>

In [4]:
#update symbology of geocoded address
arcpy.management.ApplySymbologyFromLayer(
    in_layer="NewCharterLocationPoint",
    in_symbology_layer="IngeniumCharter",
    symbology_fields=None,
    update_symbology="MAINTAIN"
)

#map = aprx.listMaps()[0]
#for lyr in map.listLayers():
    #print(lyr.name)
    #if "NewCharterLoca_GeocodeAddres" in lyr.name:
        #layerName = str(lyr.name)
        #lyr.name = lyr.name.replace(layerName, school_name )
        #print(f"Layer renamed to: {lyr.name}")

<Result 'NewCharterLocationPoint'>

# Make Buffer Zone

In [5]:
#create buffer of 3-miles from layer
#change in_features to school_name
arcpy.analysis.Buffer(
    in_features="NewCharterLocationPoint",
    out_feature_class=r"C:\Users\Avigail.Vargas\Documents\ArcGIS\Projects\PersonalProjects\3MileRadiusTemplate\Default.gdb\New_3MileBufferFill",
    buffer_distance_or_field="3 Miles",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)
#change in_features to school_name
arcpy.analysis.Buffer(
    in_features="NewCharterLocationPoint",
    out_feature_class=r"C:\Users\Avigail.Vargas\Documents\ArcGIS\Projects\PersonalProjects\3MileRadiusTemplate\Default.gdb\New_BufferBorder",
    buffer_distance_or_field="3 Miles",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)

arcpy.management.ApplySymbologyFromLayer(
    in_layer="New_3MileBufferFill",
    in_symbology_layer="Ingenium_3MileBufferFill",
    symbology_fields=None,
    update_symbology="MAINTAIN"
)

arcpy.management.ApplySymbologyFromLayer(
    in_layer="New_BufferBorder",
    in_symbology_layer="Ingenium_3MileBuffer",
    symbology_fields=None,
    update_symbology="MAINTAIN"
)

<Result 'New_BufferBorder'>

# Update Queries to Desired Config

In [6]:
#updates config to 6-8. Excludes most magnet centers.
query_polys = f"CONFIG IN ('4-12', '4-8', '6-12', '6-8', '7-12', '7-8', 'K-8', 'TK-8', 'PK-12', 'TK-12', '1-6', 'K-10', 'K-12', 'K-6', 'TK-6')"
query_ptspoly = f"CONFIG IN ('4-12', '4-8', '6-12', '6-10', '5-8', '6-7', '6-8', '7-12', '7-8', 'K-8', 'K-7', 'PK-12', 'TK-12', 'TK-8', '1-6', '2-6', '3-6', '4-6', '6-6', 'K-6', 'TK-6') And LausdPts_Allsites.BSYS_CLAS NOT IN ('Magnet Center - Elementary', 'Magnet Center - Middle School', 'Magnet Center - Senior High')"

l = m.listLayers("LAUSDPolys")[0]

if l.supports('DefinitionQuery'):
  #Gets the list of definition queries
  dql = l.listDefinitionQueries()
  #Clears active definition queries otherwise the update will fail if there is already an active query
  for dq in dql:
    dq['isActive'] = False
  #Creates a new definition query and append it to the list
  dql.append({'name': 'Appended Query', 'sql': query_polys, 'isActive': True})
  #Updates the definition queries with the newly modified list
  l.updateDefinitionQueries(dql)
 
l = m.listLayers("LausdPtsMapPoly")[0]

if l.supports('DefinitionQuery'):
  #Gets the list of definition queries
  dql = l.listDefinitionQueries()
  #Clears active definition queries otherwise the update will fail if there is already an active query
  for dq in dql:
    dq['isActive'] = False
  #Creates a new definition query and append it to the list
  dql.append({'name': 'Appended Query', 'sql': query_ptspoly, 'isActive': True})
  #Updates the definition queries with the newly modified list
  l.updateDefinitionQueries(dql)

# Creating Excel List in Buffer Zone

In [10]:
#select by location all polygons and points that intersect buffer zone
arcpy.management.SelectLayerByLocation(
    in_layer="LausdPtsMapPoly;LausdPolys",
    overlap_type="INTERSECT",
    select_features="New_3MileBufferFill",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#export table of selected features as an excel sheet
arcpy.conversion.TableToExcel(
    Input_Table="LausdPolys;LausdPtsMapPoly",
    Output_Excel_File=r"C:\PersonalProjects\ExcelSchoolsConfig6_8.xls",
    Use_field_alias_as_column_header="ALIAS",
    Use_domain_and_subtype_description="CODE"
)
#clear selection
arcpy.management.SelectLayerByAttribute("LausdPolys","CLEAR_SELECTION")
arcpy.management.SelectLayerByAttribute("LausdPtsMapPoly","CLEAR_SELECTION")
#Get COS
#Change select_features to school_name
arcpy.management.SelectLayerByLocation(
    in_layer="COS_Boundary",
    overlap_type="INTERSECT",
    select_features="NewCharterLocationPoint",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CopyFeatures("COS_Boundary","New_COS_Boundary")
arcpy.management.SelectLayerByAttribute("COS_Boundary","CLEAR_SELECTION")
#Select by location all LAUSDPolygons and Points within COS
arcpy.management.SelectLayerByLocation(
    in_layer="LausdPtsMapPoly;LausdPolys",
    overlap_type="INTERSECT",
    select_features="New_COS_Boundary",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
#Export selected features as an Excel sheet
arcpy.conversion.TableToExcel(
    Input_Table="LausdPolys;LausdPtsMapPoly",
    Output_Excel_File=r"C:\PersonalProjects\ExcelCOSConfig6_8.xls",
    Use_field_alias_as_column_header="ALIAS",
    Use_domain_and_subtype_description="CODE"
)
#clear selection
arcpy.management.SelectLayerByAttribute("New_COS_Boundary","CLEAR_SELECTION")
arcpy.management.SelectLayerByAttribute("LausdPolys","CLEAR_SELECTION")
arcpy.management.SelectLayerByAttribute("LausdPtsMapPoly","CLEAR_SELECTION")

<Result 'New_COS_Boundary'>